In [1]:
import numpy as np
import scipy.stats

In [2]:
import tensorflow as tf

In [3]:
M = 5
N = 2
T = 3
h = 2
s = 3

In [4]:
A_np = np.random.randn(M, h)
C_np = np.random.randn(s, T)
B_np = np.random.randn(h, N, s)

In [5]:
print(A_np)
print(B_np)
np.tensordot(A_np, B_np, axes=1)

[[ 0.05825791  1.48698437]
 [-1.43003367  0.71842057]
 [-0.42091422 -0.4881037 ]
 [-0.05220967  1.14356714]
 [-0.6310334  -0.04739766]]
[[[-0.38047844 -1.07159392 -0.15721609]
  [-1.27471406  0.38387734 -1.84856537]]

 [[-1.31405464 -0.84038003 -0.19862652]
  [ 0.63067744 -0.93616276  0.61569205]]]


array([[[-1.9761446 , -1.31206081, -0.30451361],
        [ 0.86354531, -1.3696955 ,  0.8078309 ]],

       [[-0.39994689,  0.92866909,  0.08212693],
        [ 2.27597567, -1.2215161 ,  3.08583656]],

       [[ 0.80154372,  0.86124173,  0.16312483],
        [ 0.22870928,  0.29536508,  0.47756589]],

       [[-1.48284504, -0.90508342, -0.21893456],
        [ 0.7877744 , -1.09060707,  0.80059819]],

       [[ 0.30237773,  0.71604361,  0.10862304],
        [ 0.77449451, -0.1978675 ,  1.13732413]]])

In [6]:
A_tf = tf.Variable(A_np)
C_tf = tf.Variable(C_np)
B_tf = tf.Variable(B_np)

In [7]:
AB_tf = tf.tensordot(A_tf, B_tf,axes = [[1], [0]])
sess = tf.Session()
sess.run(tf.global_variables_initializer())
AB = sess.run(AB_tf)

In [8]:
print(AB)

[[[-1.9761446  -1.31206081 -0.30451361]
  [ 0.86354531 -1.3696955   0.8078309 ]]

 [[-0.39994689  0.92866909  0.08212693]
  [ 2.27597567 -1.2215161   3.08583656]]

 [[ 0.80154372  0.86124173  0.16312483]
  [ 0.22870928  0.29536508  0.47756589]]

 [[-1.48284504 -0.90508342 -0.21893456]
  [ 0.7877744  -1.09060707  0.80059819]]

 [[ 0.30237773  0.71604361  0.10862304]
  [ 0.77449451 -0.1978675   1.13732413]]]


In [9]:
AB_tf_C_tf = tf.tensordot(AB_tf, C_tf, axes=[[2], [1]])
ABC = sess.run(AB_tf_C_tf)

In [10]:
print(ABC)

[[[-0.69020712  1.61714288  4.21016331]
  [-1.26572594  0.62281008  2.92806464]]

 [[ 0.8715957   0.03801363 -1.85881149]
  [-1.25585848  1.69539571  2.55832308]]

 [[ 0.55860705 -0.73563695 -2.4546514 ]
  [ 0.2379332   0.17980421 -0.64601213]]

 [[-0.45063165  1.19421437  2.97883543]
  [-1.01627589  0.57174862  2.32791957]]

 [[ 0.5419611  -0.37226233 -1.81391252]
  [-0.23767363  0.58184392  0.4001349 ]]]


In [11]:
def f(theta, x, returngrad=False):
    fval = (theta[0])*(theta[1] - x)
    if returngrad==False:
        return fval
    else:
        if (np.isscalar(x)):
            xshape = ()
        else:
            xshape = x.shape
        dfdtheta = np.zeros(theta.shape + xshape)
        # want this to broadcast correctly regardless of whether x is a matrix or vector
        dfdtheta[0,...] = fval/theta[0]
        dfdtheta[1,...] = theta[0]
        dfdtheta[2,...] = 0
        return fval, dfdtheta
    
def g(theta, x, returngrad=False):
    if (np.isscalar(x)):
        xshape = ()
        gval = theta[2]
    else:
        xshape = x.shape
        gval = np.ones(xshape)*theta[2]
        
    if returngrad==False:
        return gval
    else:
        dgdtheta = np.zeros(theta.shape + xshape)
        # want this to broadcast correctly regardless of whether x is a matrix or vector
        dgdtheta[0,...] = 0
        dgdtheta[1,...] = 0
        dgdtheta[2,...] = 1
        return gval, dgdtheta

In [12]:
theta = 0.5
omtheta = 1.-theta
alpha = np.zeros(2)
alpha[0] = 0.5/(theta*omtheta)
alpha[1] = alpha[0]*(omtheta*omtheta + theta*theta)

In [13]:
M = 10
k = 0.5
grid = np.arange(-M,(M+1))*k
yim1 = grid.copy()
h = 0.5

In [14]:
# play with sparse tensors a bit
N = 2*M+1
ind0 = np.arange(0,N)
ind = np.repeat(np.array([ind0]).T,3,axis=1)
val = np.repeat(1,N)
ds = [N,N,N]

In [15]:
test = tf.SparseTensor(indices=ind,values=val,dense_shape=ds)

In [16]:
testsparse = sess.run(testd)

NameError: name 'testd' is not defined

In [17]:
col1 = np.ones((N,1))
colr = np.array([np.arange(0,N)]).T

In [18]:
# create the sparse A tensor
ds = [N,N,N]
val = []
ind = np.zeros((0,3))
tol = N
th = np.array([1.,0.5,0.75])
col1 = np.ones((N,1))
colr = np.array([np.arange(0,N)]).T
for i in range(N):
    for j in range(N):
        if (abs(i-j) > tol):
            continue
            
        yi = grid[i]
        ystar = grid[j]
        mv = ystar + (alpha[0]*f(th,ystar) - alpha[1]*f(th,yim1))*omtheta*h
        g2star = np.square(g(th,ystar))
        g2m1 = np.square(g(th,yim1))
        vv = np.maximum(0, alpha[0]*g2star - alpha[1]*g2m1)*omtheta*h
        thesevals = scipy.stats.norm.pdf(x=yi, loc=mv, scale=np.sqrt(vv))
        indblock = np.hstack([i*col1,j*col1,colr])
        ind = np.vstack([ind, indblock])
        val.append(thesevals.tolist())

In [19]:
val = np.array(val).flatten()

In [20]:
test = tf.SparseTensor(indices=ind,values=val,dense_shape=ds)

In [21]:
testsparse = sess.run(test)